**TODO**

* get Jacobian working
* QP on arm

In [2]:
using RigidBodyDynamics
using StaticArrays
using RigidBodyDynamics.OdeIntegrators
using Plots
gr()

Plots.GRBackend()

In [3]:
# Set up mechanism

world = RigidBody{Float64}("world")
g = -9.81 # gravitational acceleration in z-direction
twolinkarm = Mechanism(world; gravity = SVector(0, 0, g))

# Parameters 
axis = SVector(0., 1., 0.) # joint axis
I_1 = 0.333 # moment of inertia about joint axis
c_1 = -0.5 # center of mass location with respect to joint axis
m_1 = 1. # mass

l_1 = -1. # length of the upper link
I_2 = 0.333 # moment of inertia about joint axis
c_2 = -0.5 # center of mass location with respect to joint axis
m_2 = 1. # mass

# Add links
frame1 = CartesianFrame3D("upper_link") # the reference frame in which the spatial inertia will be expressed
inertia1 = SpatialInertia(frame1, I_1 * axis * axis.', m_1 * SVector(0, 0, c_1), m_1)
link1 = RigidBody(inertia1)
shoulder = Joint("shoulder", Revolute(axis))
beforeShoulderToWorld = eye(Transform3D, frame_before(shoulder), default_frame(world))
attach!(twolinkarm, world, shoulder, beforeShoulderToWorld, link1)

inertia2 = SpatialInertia(CartesianFrame3D("lower_link"), I_2 * axis * axis.', m_2 * SVector(0, 0, c_2), m_2)
link2 = RigidBody(inertia2)
elbow = Joint("elbow", Revolute(axis))
beforeElbowToAfterShoulder = Transform3D(frame_before(elbow), shoulder.frameAfter, SVector(0, 0, l_1))
attach!(twolinkarm, link1, elbow, beforeElbowToAfterShoulder, link2)

Spanning tree:
Vertex: world (root)
  Vertex: upper_link, Edge: shoulder
    Vertex: lower_link, Edge: elbow
No non-tree joints.

In [4]:
# Set initial state
state = MechanismState{Float64}(twolinkarm)
configuration(state, shoulder)[:] = 0.
configuration(state, elbow)[:] = 0.0
velocity(state, shoulder)[:] = 0.
velocity(state, elbow)[:] = 0.;

setdirty!(state)

In [5]:
function controller(state)
    # TODO add calculations of required tau - try PID
    q = configuration(state)
    v = velocity(state)
    kp = 1.5; kd = 2*sqrt(kp);
    pid(cur, des) = kp.*(des[1]-cur[1]) + kd.*(des[2]-cur[2])
    
    j1_des = [0,0]
    j2_des = [0,0]
    tau = [pid([q[1],v[1]], j1_des), pid([q[2],v[2]], j2_des)]
end

result = DynamicsResult{Float64}(twolinkarm)
function controlled_dynamics!(vd::AbstractArray, sd::AbstractArray, t, state)
    tau = controller(state)
    dynamics!(result, state, tau)
    copy!(vd, result.v̇)
    copy!(sd, result.ṡ) 
end

dyn! = controlled_dynamics!
# const dyn! = damped_dynamics!
# sink = DrakeVisualizerSink(vis)
sink = ExpandingStorage{Float64}(10000)
integrator = MuntheKaasIntegrator(dyn!, runge_kutta_4(Float64), sink);

In [6]:
function plot_two_link_arm(q)
    # TODO figure out transforms in RigidBodyDynamics to allow plotting of arbitrary trees
    theta1 = q[1]-pi/2
    theta2 = q[2]
    shoulder = [0,0]
    elbow = shoulder + 1*[cos.(theta1), sin.(theta1)]
    hand = elbow + 1*[cos.(theta1+theta2), sin.(theta1+theta2)]
    
    plot([0,elbow[1]],[0,elbow[2]], linewidth=5, marker=(:circle,10), color=:orange, legend=:none, xlim=(-2.1,2.1), ylim=(-2.1,2.1))
    plot!([elbow[1], hand[1]],[elbow[2], hand[2]], linewidth=5, marker=(:circle,10), color=:orange)
    plot!(xlabel="x", ylabel="z")
end

plot_two_link_arm (generic function with 1 method)

In [74]:
# Initial conditions
configuration(state, shoulder)[:] = 0.3
configuration(state, elbow)[:] = 0.4
velocity(state, shoulder)[:] = 0.
velocity(state, elbow)[:] = 0.;

anim = Animation()
integrate(integrator, state, 5.0, 1e-3, maxRealtimeRate = Inf)
qs = sink.qs
for i in 1:100:length(qs)
    plot_two_link_arm(qs[i])
    frame(anim)
end 

gif(anim, "img/arm_random.gif")

INFO: Saved animation to /Users/Kazu/Code/julia/JuliaExperiments/img/arm_random.gif


Plots.AnimatedGif("/Users/Kazu/Code/julia/JuliaExperiments/img/arm_random.gif")

In [8]:
nv = num_velocities(state)

bodyframe = default_frame(link2)
baseframe = default_frame(world)
jacobian = GeometricJacobian(bodyframe, baseframe, baseframe, Matrix{Float64}(3, nv), Matrix{Float64}(3, nv))
p = path(twolinkarm, link2, world)
geometric_jacobian!(jacobian, state, path(twolinkarm, world, link2))
jacobian.linear

3×2 Array{Float64,2}:
 0.0   0.999639 
 0.0   0.0      
 0.0  -0.0268753

In [9]:
using ForwardDiff

# Manual jacobian
function handpos_from_config(q::Vector)
    [cos(q[1]-pi/2)+cos(q[1]-pi/2+q[2]), 0, sin(q[1]-pi/2)+sin(q[1]-pi/2+q[2])]
end

J_hand = x -> ForwardDiff.jacobian(handpos_from_config,x)
J_hand([0,0])

3×2 Array{Float64,2}:
 2.0          1.0        
 0.0          0.0        
 1.22465e-16  6.12323e-17

In [78]:
center_of_mass(state)

configuration(state, shoulder)[:] = pi/2
configuration(state, elbow)[:] = pi/2

transform_to_root(state, elbow.frameAfter)
# mass_matrix(state)

Transform3D from "after_elbow" to "world":
rotation: 0.6999934072230988 rad about [1.53644e-8, 1.0, 0.0], translation: [-0.295521, 0.0, -0.955336]

In [79]:
plot_two_link_arm(state.q)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 x 
 
 
 z

In [11]:
M = result.massmatrix

2×2 Symmetric{Float64,Array{Float64,2}}:
 2.66594   0.832972
 0.832972  0.333   

In [12]:
N = result.dynamicsbias

2-element Array{Float64,1}:
 0.579272
 0.183931

In [14]:
using JuMP, Ipopt

In [81]:
# Constants
Δt = 0.1
num_time_steps = 20
max_acceleration = 0.5
max_torque = 10
kp = 1.5
kd = 2*sqrt(1.)

function qp_controller(q::Array, q̇::Array, p_des::Array)
    model = Model(solver=IpoptSolver(print_level=0))
    
    # Decision variables
    @variables model begin
        -max_acceleration <= q̈[1:2] <= max_acceleration
        -max_torque <= τ[1:2] <= max_torque
    end
    
    # TODO dynamics constraints, incorporate torque constraints?
    M = result.massmatrix
    N = result.dynamicsbias
    @constraint(model, M*q̈+N .== τ)
    
    # Cost function: spring damper behavior (PD controller) on \ddot{p} between current and goal
    # TODO try converting this to Q, C matrices
    J = J_hand(q)
    Q = J'*J
    # TODO calculate p and v
    p = handpos_from_config(q)
    v = J*q̇
#     c = kp*(p_des-p) - kd*v # TODO add jacobian_dot
    c = kp*(p_des[[1,3]]-p[[1,3]]) - kd*v[[1,3]]
    
    @objective(model, Min, q̈'*Q*q̈ + c'*q̈)

    solve(model)
    return getvalue(q̈)
end

qp_controller (generic function with 2 methods)

In [82]:
    model = Model(solver=IpoptSolver(print_level=0))
    
    # Decision variables
    @variables model begin
        -max_acceleration <= q̈[1:2] <= max_acceleration
        -max_torque <= τ[1:2] <= max_torque
    end
    
    # TODO dynamics constraints, incorporate torque constraints?
    M = result.massmatrix
    N = result.dynamicsbias
    @constraint(model, M*q̈+N .== τ)
    
    # Cost function: spring damper behavior (PD controller) on \ddot{p} between current and goal
    # TODO try converting this to Q, C matrices
    J = J_hand(q)
    Q = J'*J
#     # TODO calculate p and v
    p = handpos_from_config(q)
    v = J*q̇
    p_des = [0,0,0]
    c = kp*(p_des[[1,3]]-p[[1,3]]) - kd*v[[1,3]] # TODO add jacobian_dot
    
#     @objective(model, Min, q̈'*Q*q̈ + c'*q̈)


2-element Array{Float64,1}:
 -1.52628
 -1.50275

In [83]:
test = [1,2,3]
test[[1,3]]

2-element Array{Int64,1}:
 1
 3

In [84]:
q = configuration(state)
q̇ = velocity(state)

2-element Array{Float64,1}:
  0.00137477
 -0.0145133 

In [86]:
qp_controller(q, q̇, [0.0, 0.0, 0.0])

2-element Array{Float64,1}:
 0.131569
 0.5     